<a href="https://colab.research.google.com/github/jamstanleyambe/End-to-End-Dog-Breed-Identification/blob/main/Dog_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End-to-End multil-class Dog Breed Classification

This NoteBook builds an end to end multil-class image classifier using TensorFlow and TensorFlow Hub.

## 1. Problem


> Identifying the bread of a dog given an image of a dog.
if i can take a picture of any dog i want to know what kind of the breed the Dog is.

## 2. Data

>The data we are using is coming from Kaggle dog breed identification competition

## 3. Evaluation

> The Evaluation is a file with prediction probability of each dog breed of each test image

## 4. Features

>Some information about the data:
 * We are dealing with images **(unstructured data)** so it probably best to use deep learning/transfer learning.
 * There are 120 breeds of dogs (this means thee are 120 different classes).
 * There are around 10000 + images in the traing set ( this images has lable beacuse we are using them for the trainig)
 * Tehre are around 10000 + images in the test set, (this images has no lable because we want to predict them)


In [1]:
50*50

2500